# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [11]:
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution, easy_datetime_distribution
import sys
import os
sys.path.append(os.path.realpath('../'))
from medusa.edge_extractor import *
from medusa.matrix_modifier import MatrixModifier
from medusa.medusa_lca import *
from medusa.timeline_builder import TimelineBuilder
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [12]:
from tests.databases import *
db_electrolysis()

100%|██████████| 1/1 [00:00<00:00, 32263.88it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 52428.80it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 48770.98it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 3/3 [00:00<00:00, 77195.78it/s]

Vacuuming database 


In [13]:
demand = {('foreground', 'A'): 1}
gwp = ('GWP', 'example')

In [14]:
SKIPPABLE = [node.id for node in bd.Database('background_2008')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

In [15]:
import warnings
warnings.filterwarnings("ignore")

database_date_dict = {
            'background_2008': datetime.strptime("2008", "%Y"),
            'background_2024': datetime.strptime("2024", "%Y"),
            'foreground': "dynamic", # flag databases that should be temporally distributed with "dynamic"
        }

In [16]:
# mlca = MedusaLCA(demand, gwp, filter_function, database_date_dict, max_calc=10)
mlca = MedusaLCA(demand, gwp, filter_function, database_date_dict, max_calc=1000)

Starting graph traversal
Calculation count: 4


In [17]:
mlca.build_timeline()

All databases in database_date_dict exist as brightway project databases


,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2018,9,2018-01-01,2,Electricity mix,2019,11,2019-01-01,6,"Hydrogen production, electrolysis",1.0,{'background_2020': 1}
1,2019,10,2019-01-01,2,Electricity mix,2019,11,2019-01-01,6,"Hydrogen production, electrolysis",3.0,{'background_2020': 1}
2,2019,11,2019-01-01,6,"Hydrogen production, electrolysis",2022,16,2022-01-01,7,"Hydrogen production, electrolysis2",13.0,{'background_2020': 1}
3,2020,12,2020-01-01,2,Electricity mix,2019,11,2019-01-01,6,"Hydrogen production, electrolysis",1.0,{'background_2020': 1}
4,2020,12,2020-01-01,2,Electricity mix,2021,14,2021-01-01,6,"Hydrogen production, electrolysis",1.0,{'background_2020': 1}
5,2021,13,2021-01-01,2,Electricity mix,2021,14,2021-01-01,6,"Hydrogen production, electrolysis",3.0,"{'background_2020': 0.7494866529774127, 'backg..."
6,2021,14,2021-01-01,6,"Hydrogen production, electrolysis",2024,17,2024-01-01,7,"Hydrogen production, electrolysis2",13.0,"{'background_2020': 0.7494866529774127, 'backg..."
7,2022,15,2022-01-01,2,Electricity mix,2021,14,2021-01-01,6,"Hydrogen production, electrolysis",1.0,"{'background_2020': 0.4996577686516085, 'backg..."
8,2022,16,2022-01-01,7,"Hydrogen production, electrolysis2",2024,18,2024-01-01,8,"Heat production, hydrogen",0.63,"{'background_2020': 0.4996577686516085, 'backg..."
9,2024,17,2024-01-01,7,"Hydrogen production, electrolysis2",2024,18,2024-01-01,8,"Heat production, hydrogen",0.07,{'background_2024': 1}


In [18]:
mlca.build_datapackage()

In [19]:
mlca.lci()

In [20]:
mlca.lcia()

In [21]:
print('New MEDUSA LCA Score:', mlca.score)
print('Old static LCA Score:', mlca.static_lca.score)

New MEDUSA LCA Score: 45.49999922513962
Old static LCA Score: 45.49999922513962


In [22]:
reversed_dict = {value: key for key, value in mlca.time_mapping_dict.items()}


In [23]:
df = pd.DataFrame(mlca.technosphere_matrix.toarray())
df.rename(mlca.dicts.activity.reversed, inplace=True, axis=0)
df.rename(mlca.dicts.activity.reversed, inplace=True, axis=1)
df.rename(reversed_dict, inplace=True, axis=0)
df.rename(reversed_dict, inplace=True, axis=1)
df

,"((background_2024, electricity_mix), 2024)","((background_2024, electricity_wind), 2024)","((background_2020, electricity_mix), 2020)","((background_2020, electricity_wind), 2020)","((foreground, electrolysis), dynamic)","((foreground, electrolysis2), dynamic)","((foreground, heat_from_hydrogen), dynamic)","((exploded, electricity_mix), 2018)","((exploded, electricity_mix), 2019)","((exploded, electrolysis), 2019)","((exploded, electricity_mix), 2020)","((exploded, electricity_mix), 2021)","((exploded, electrolysis), 2021)","((exploded, electricity_mix), 2022)","((exploded, electrolysis2), 2022)","((exploded, electrolysis2), 2024)","((exploded, heat_from_hydrogen), 2024)"
"((background_2024, electricity_mix), 2024)",1.0,0.0,0.0,0.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.250513,0.0,-0.500342,0.0,0.0,0.00
"((background_2024, electricity_wind), 2024)",-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00
"((background_2020, electricity_mix), 2020)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,-0.749487,0.0,-0.499658,0.0,0.0,0.00
"((background_2020, electricity_wind), 2020)",0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00
"((foreground, electrolysis), dynamic)",0.0,0.0,0.0,0.0,1.0,-13.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00
"((foreground, electrolysis2), dynamic)",0.0,0.0,0.0,0.0,0.0,1.0,-0.7,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00
"((foreground, heat_from_hydrogen), dynamic)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00
"((exploded, electricity_mix), 2018)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00
"((exploded, electricity_mix), 2019)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-3.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00
"((exploded, electrolysis), 2019)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,-13.0,0.0,0.00
